# Lab 1: Problem 1 (MDP algorithms: tabular Q-learning)

*OpenAI gym FrozenLake environment*

Winter is here. You and your friends were tossing around a frisbee at the park
    when you made a wild throw that left the frisbee out in the middle of the lake.
    The water is mostly frozen, but there are a few holes where the ice has melted.
    If you step into one of those holes, you'll fall into the freezing water.
    At this time, there's an international frisbee shortage, so it's absolutely imperative that
    you navigate across the lake and retrieve the disc.
    However, the ice is slippery, so you won't always move in the direction you intend.
    The surface is described using a grid like the following

        SFFF
        FHFH
        FFFH
        HFFG

    S : starting point, safe
    F : frozen surface, safe
    H : hole, fall to your doom
    G : goal, where the frisbee is located

    The episode ends when you reach the goal or fall in a hole.
    You receive a reward of 1 if you reach the goal, and zero otherwise.
    
    FrozenLake-v0 defines "solving" as getting average reward of 0.78 over 100 consecutive trials.


In [1]:
#wandb set up for logging runs online and moving them to the leaderboard
import wandb
wandb.login()
run=wandb.init(project="lab1", tags=["problem1"], entity="ieor-4575")

ModuleNotFoundError: No module named 'wandb'

In [2]:
## DO NOT CHANGE THIS CELL
import numpy as np
import gym
env=gym.make('FrozenLake-v0')
env.seed(0)

[0]

For proper accounting rewards while you learn, we build a wrapper around env.step() and env.reset(). In an episode, every time you take an action the reward will be appended to the reward of the episode, and whenever the environment is reset (at the end of an epsiode), the episode reward is reset to 0. 

In [3]:
## DO NOT CHANGE THIS CELL
#wrapper for accounting rewards
rEpisode=0
rList=[]
fixedWindow=100
movingAverage=0

def reset_decorate(func):
    def func_wrapper():
        global rList
        global movingAverage
        global rEpisode
        global fixedwindow
        rList.append(rEpisode)
        if len(rList) >= fixedWindow:
            movingAverage=np.mean(rList[len(rList)-fixedWindow:len(rList)-1])
        rEpisode=0
        return(func())
    return func_wrapper

env.reset = reset_decorate(env.reset)

def step_decorate(func):
    def func_wrapper(action):
        global rEpisode
        s1, r, d, other = func(action)
        rEpisode+=r
        return(s1, r, d, other)
    return func_wrapper

env.step = step_decorate(env.step)

def init():
    rEpisode=0
    rList=[]
    movingAverage=0
    return

Below we illustrate the execution of the Open AI gym enviornment using the policy of chosing random action in every state. Every time an action is taken the enviorment returns a tuple containing next state, reward, and the status (whether terminal state is reached or not). 

In [4]:
### RANDOM SAMPLING EXAMPLE
num_episodes=1000; #number of episodes you want to try
episode_max_length=100; #you can explicitly end the episode before terminal state is reached

env.reset()
#env.render()
#execute in episodes
for i in range(num_episodes):
    
    #reset the environment at the beginning of an episode
    s = env.reset()
    d = False #not done
    
    for t in range(episode_max_length):
        
        ################ Random action policy ###########################
        #play random action 
        a = env.action_space.sample()
        #get new state, reward, done
        s, r, d, _ = env.step(a)
        #################################################################
        
        
        #break if done, reached terminal state 
        if d == True:
            break
    
    
    #log per-episode reward and moving average over 100 episodes
    # wandb.log({ "random reward" : rEpisode, "random reward moving average" : movingAverage, "random episode" : i})

Implement tabular Q-learning (*YOU SHOULD ONLY CHANGE THE CELL BELOW*)

In [5]:
"""
desert_totem813 was a good run

ideas:

gamma: 
    high gamma: bot looks far into future is good bc rewards are sparse

alpha: 
    decays as the robot learns i.e. with more episodes

penalty for falling in the ice:
    fixes sparseness problems, but environment may be too stochastic
    code:
        if d==True and s1 != 15:
        r = - penalty_value
"""

#initialize episodic structure
init()
num_episodes=1000; #init 1000
episode_max_length=100; #init 100

def gamma(i, g=.95):
    return g

def alpha(i, learnRate=0.8): 
    # 1, .99, .98, .97 ,...., 0
    return 1 - ((i) / num_episodes)
    
def epsilon(i):
    return 1 - ((i) / num_episodes)
    # return 1 / np.sqrt(i+1)
    
Q=np.zeros([env.observation_space.n,env.action_space.n]) #Q(s,a). The Q-values from this array will be used to evaluate your policy.
n=np.ones([env.observation_space.n,env.action_space.n])  # recording number of trails for each arm

for i in range(num_episodes):
    #reset the environment at the beginning of an episode
    s = env.reset()
    d = False #not done
    
    learning_rate = alpha(i)
    discount = gamma(i)
    exploration = epsilon(i)
    
    for t in range(episode_max_length):
        
        """ 
        epsilon decision 
        exploration incentive decays as episodes increase bc Q values converge
        want e s.t. e-greedy -> greedy as i -> inf
        i.e.
        want e s.t. e~~0 when i == num_episodes
        """
        p = np.random.rand()
        if p < exploration:
            a = env.action_space.sample()
        else:
            a = np.argmax(Q[s,:])
        
        #get new state, reward, done
        s1, r, d, info = env.step(a)
        
        # lets penalize the holes
        if d==True and s1 != 15:
            r = - 1
        
        ##### update Q(s,a) ############
        delta = r + discount*max(Q[s1,:]) - Q[s,a]
        Q_update = Q[s,a] + learning_rate*delta
        Q[s,a] = Q_update
        
        #break if done, reached terminal state 
        if d == True:
            break
        s=s1
        
    #log per-episode reward and moving average over 100 episodes
    wandb.log({ "training reward" : rEpisode, "training reward moving average" : movingAverage, "training episode" : i})
wandb.run.summary["number of training episodes"]=num_episodes

In [6]:
%%wandb
## DO NOT CHANGE THIS CELL. CHANGING ANY PART OF THIS CELL CAN DISQUALIFY THE SUBMISSION
#Evaluation of trained policy
init()
num_episodes=1000; #number of episodes for evaluation
episode_max_length=100; 
movingAverageArray=[]
score=0
env.reset()
for i in range(num_episodes):
    s = env.reset()
    d = False #not done
    for t in range(episode_max_length):
        a = np.argmax(Q[s,:])
        s, r, d, _ = env.step(a)
        if d == True:
            break
    #log per-episode reward and moving average over 100 episodes
    wandb.log({ "evaluation reward" : rEpisode, "evaluation reward moving average" : movingAverage, "evaluation episode" : i})
    movingAverageArray.append(movingAverage)
    #score is x if there is a window of 100 consecutive episodes where moving average was at least x
    if i>100:
        score=max(score,min(movingAverageArray[i-100:i-1]))


wandb.run.summary["score"]=score 

In [7]:
print(score)

0.7575757575757576


In [8]:
run.finish()

training reward,0.0
training reward moving average,0.55556
training episode,999
_step,1999
_runtime,3
_timestamp,1613966167
number of training episodes,1000
evaluation reward,1.0
evaluation reward moving average,0.72727
evaluation episode,999
score,0.75758


training reward,▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█
training reward moving average,▁▁▁▁▁▁▁▁▁▂▂▁▁▁▁▁▁▂▁▂▁▁▁▂▂▂▃▃▃▃▃▄▄▄▄▄▅▆▇█
training episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█████████████████
evaluation reward,▁██▁█▁█▁████▁▁█▁███▁████████▁▁█████████▁
evaluation reward moving average,▁▅▆▇▇▇▇██▇▆▆▇█▆▇▅▅▆▅▅▅▆▇▇▇▆▆▇▆▆▆▆▅▇▇▅▆▆▅
evaluation episode,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
